# Load Data

In [1]:
import hickle  as hkl

In [2]:
data2 =hkl.load('./Gen_data/CROSS_Net_input.hkl')

# Generate LSTM NetWork

In [3]:
import theano

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [321]:
import lasagne

In [11]:
import numpy as np

In [4]:
from lasagne.layersgne.layers import InputLayer
from lasagne.layers import LSTMLayer
from lasagne.layers import ReshapeLlasagne
from lasagne.layers import DenseLayer

In [13]:
from lasagne.updates import adam
from lasagne.objectives import squared_error

from nolearn.lasagne import NeuralNet
from nolearn.lasagne import TrainSplit

In [327]:
# inititalization gate
gate_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
    b=lasagne.init.Constant(0.))

cell_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
    # Setting W_cell to None denotes that no cell connection will be used.
    W_cell=None, b=lasagne.init.Constant(0.),
    # By convention, the cell nonlinearity is tanh in an LSTM.
    nonlinearity=lasagne.nonlinearities.tanh)

In [330]:
minibatch,cutdown_len,num_units, = 13, 13, 5
N_HIDDEN=10
l_in = InputLayer(shape=(128,minibatch,1))  
layer_mask = InputLayer(shape=(128,minibatch))
layer2 = LSTMLayer(layer1,N_HIDDEN,mask_input=layer_mask,ingate=gate_parameters,
                   cell=cell_parameters,outgate=gate_parameters,learn_init=True,grad_clipping=100.)
layer2_back = LSTMLayer(layer1,N_HIDDEN,mask_input=layer_mask,ingate=gate_parameters,  
                   cell=cell_parameters,outgate=gate_parameters,learn_init=True,grad_clipping=100.,backwards=True)
# We'll combine the forward and backward layer output by summing.
# Merge layers take in lists of layers to merge as input.
l_sum = lasagne.layers.ElemwiseSumLayer([layer2, layer2_back])

In [331]:
# First, retrieve symbolic variables for the input shape
n_batch, n_time_steps, n_features = l_in.input_var.shape
# Now, squash the n_batch and n_time_steps dimensions
l_reshape = lasagne.layers.ReshapeLayer(l_sum, (-1, N_HIDDEN))
# Now, we can apply feed-forward layers as usual.
# We want the network to predict a single value, the sum, so we'll use a single unit.
l_dense1 = lasagne.layers.DenseLayer(
    l_reshape, num_units=5, nonlinearity=lasagne.nonlinearities.tanh)
network = lasagne.layers.DenseLayer(
    l_dense1, num_units=1, nonlinearity=lasagne.nonlinearities.tanh)
# Now, the shape will be n_batch*n_timesteps, 1.  We can then reshape to
# n_batch, n_timesteps to get a single value for each timstep from each sequence
l_out = lasagne.layers.ReshapeLayer(network, (n_batch, n_time_steps))

In [338]:

# Symbolic variable for the target network output.
# It will be of shape n_batch, because there's only 1 target value per sequence.
target_values = theano.tensor.vector('target_output')

# lasagne.layers.get_output produces an expression for the output of the net
network_output = lasagne.layers.get_output(l_out)
# The value we care about is the final value produced for each sequence
# so we simply slice it out.
predicted_values = network_output[:, -1]
# Our cost will be mean-squared error
cost =  theano.tensor.mean((predicted_values - target_values)**2)
# Retrieve all parameters from the network
all_params = lasagne.layers.get_all_params(l_out)
# Compute adam updates for training
updates = lasagne.updates.adam(cost, all_params)
# Theano functions for training and computing cost
train = theano.function(
    [l_in.input_var, target_values, layer_mask.input_var],cost, updates=updates)
compute_cost = theano.function( [l_in.input_var, target_values, layer_mask.input_var], cost)

MissingInputError: An input of the graph, used to compute InplaceDimShuffle{1,0,2}(input), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.

Backtrace when the variable is created:
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 390, in execute_request
    user_expressions, allow_stdin)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/zmqshell.py", line 501, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/yinqijin/anaconda2/lib/python2.7/s

In [336]:
# We'll train the network with 10 epochs of 100 minibatches each
NUM_EPOCHS = 100
EPOCH_SIZE = 10
m = ones(minibatch,1)
print m
for epoch in range(NUM_EPOCHS):
    for _ in range(EPOCH_SIZE):
        train(X_train, y_train, m)
    #if epoch%10==0:
    cost_val = compute_cost(X_test, y_test, m)
    print("Epoch {} validation cost = {}".format(epoch + 1, cost_val))

NameError: name 'ones' is not defined


lr = theano.shared(np.float32(1e-4))

net = NeuralNet(
            network,
            max_epochs=50,
            update=adam,
            update_learning_rate=lr,
            train_split=TrainSplit(eval_size=0.1),
                regression = True,
            objective_loss_function = squared_error,
            #on_epoch_finished=[
            #    AdjustVariable(lr, target=1e-8, half_life=20)],
            verbose=4)

net.fit(X_train, y_train)

# Test  
网络输入使用one-hot编码，不能运行

In [160]:
cutdown_len,num_units = 13, 4
layer1 = InputLayer(shape=(None,cutdown_len,4))  #输入是序列的one-hot编码
layer2 = LSTMLayer(layer1,num_units= num_units)
layer3 = ReshapeLayer(layer2,(-1,num_units))
layer4 = DenseLayer(layer3,num_units=5 ,nonlinearity=tanh)
network = DenseLayer(layer4,num_units=1,nonlinearity=tanh)

cutdown_len = 13
layer1 = InputLayer(shape=(None,cutdown_len,4))
layer2 = DenseLayer(layer1, num_units=20 ,nonlinearity=tanh)
network = DenseLayer(layer2,num_units=1,nonlinearity=tanh)

In [161]:

lr = theano.shared(np.float32(1e-4))

net = NeuralNet(
            network,
            max_epochs=50,
            update=adam,
            update_learning_rate=lr,
            train_split=TrainSplit(eval_size=0.1),
                regression = True,
            objective_loss_function = squared_error,
            #on_epoch_finished=[
            #    AdjustVariable(lr, target=1e-8, half_life=20)],
            verbose=4)

In [162]:
X = data2['mat']
Y = data2['score']

Y_ = [ item[(len(item)-1)/2] for item in Y]
Y_ = np.array(Y_)

X_ = np.zeros((len(X),4,cutdown_len))
X_[:,:,:] = X[:]




In [163]:
from sklearn.cross_validation import ShuffleSplit
rs = ShuffleSplit(len(Y_), n_iter=1)

In [164]:
for train_idx, test_idx in rs:
    X_train = X_[train_idx,:]
    y_train = Y_[train_idx]
    X_test = X_[test_idx,:]
    y_test = Y_[test_idx]

In [165]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
print X_train[:2]
print y_train[:2]

[[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.]
  [ 0.  1.  0.  0.  0.  1.  1.  1.  0.  1.  0.  1.  1.]]

 [[ 1.  1.  0.  0.  1.  1.  0.  0.  0.  1.  0.  1.  1.]
  [ 0.  0.  1.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]]
[ 0.021  0.301]


In [166]:
net.fit(X_train, y_train)

# Neural Network with 251 learnable parameters

## Layer information

  #    name  size
---  ------  ------
  0          13x4
  1          13x5
  2          5
  3          5
  4          1



ValueError: dimension mismatch in args to gemm (512,13)x(4,20)->(512,20)
Apply node that caused the error: GpuDot22(GpuReshape{2}.0, GpuJoin.0)
Toposort index: 91
Inputs types: [CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(512, 13), (4, 20)]
Inputs strides: [(13, 1), (20, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuReshape{3}(GpuDot22.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

# Import Data

In [255]:
import numpy as np
import pandas as pd


In [256]:
temp =[]
for string in data2['seq']:
    temp.append([ item for item in string])
    

In [270]:

#j将碱基序列转化为数字序列
#d = {'a':1, 'A':1, 'g':2, 'G':2, 'c':3, 'C':3, 't':4, 'T':4, 'N':5, 'n':5}
d = {'a':1.1, 'A':1.1, 'g':3.5, 'G':3.5, 'c':5.6, 'C':5.6, 't':7.9, 'T':7.9, 'N':11, 'n':11}
func = lambda x:d[x]

X_pd = pd.DataFrame(temp)


In [271]:
X_=X_pd.applymap(func)

In [272]:
X=X_.values[:,:]
type(X)

numpy.ndarray

In [273]:
Y = data2['score']

In [274]:
Y_ = [ item[(len(item)-1)/2] for item in Y]
Y_ = np.array(Y_)



In [275]:
print X[:2]
print Y_[:2]

[[ 3.5  3.5  3.5  1.1  3.5  5.6  5.6  5.6  1.1  3.5  1.1  1.1  3.5]
 [ 3.5  3.5  1.1  3.5  5.6  5.6  5.6  1.1  3.5  1.1  1.1  3.5  3.5]]
[ 0.015  0.002]


In [276]:
from sklearn.cross_validation import ShuffleSplit
rs = ShuffleSplit(len(Y_), n_iter=1)

In [277]:
for train_idx, test_idx in rs:
    X_train = X[train_idx,:]
    y_train = Y_[train_idx]
    X_test = X[test_idx,:]
    y_test = Y_[test_idx]

In [278]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [279]:
print X_train[:2]
print y_train[:2]

[[ 1.10000002  1.10000002  1.10000002  3.5         1.10000002  5.5999999
   7.9000001   5.5999999   1.10000002  5.5999999   7.9000001   3.5
   7.9000001 ]
 [ 3.5         3.5         5.5999999   5.5999999   1.10000002  5.5999999
   1.10000002  5.5999999   7.9000001   1.10000002  1.10000002  5.5999999
   7.9000001 ]]
[ 0.31900001 -0.59399998]


# Test Network

In [141]:
net.fit(X_train, y_train)

# Neural Network with 571 learnable parameters

## Layer information

  #    name  size
---  ------  ------
  0          13x1
  1          13x10
  2          10
  3          5
  4          1



TypeError: Bad input argument to theano function with name "/home/yinqijin/.local/lib/python2.7/site-packages/nolearn/lasagne/base.py:639" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 390, in execute_request
    user_expressions, allow_stdin)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/ipykernel/zmqshell.py", line 501, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/yinqijin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", l

# Test 2 
直接用碱基序列做LSTM  

结果: 
    １．不准确
问题: 　
    1. 输入是序列，输出是概率，不是同一类，可以推出来么？
    ２. 

In [167]:
from lstm import LstmParam, LstmNetwork

class ToyLossLayer:
    """
    Computes square loss with first element of hidden layer array.
    """
    @classmethod
    def loss(self, pred, label):
        return (pred[0] - label) ** 2

    @classmethod
    def bottom_diff(self, pred, label):
        diff = np.zeros_like(pred)
        diff[0] = 2 * (pred[0] - label)
        return diff

losss =[]
def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count 
    mem_cell_ct = 100
    x_dim =  13
    concat_len = x_dim + mem_cell_ct
    lstm_param = LstmParam(mem_cell_ct, x_dim) 
    lstm_net = LstmNetwork(lstm_param)
    y_list = y_train[:1000]
    input_val_arr = X_train[:1000,:]

    for cur_iter in range(100):
        print "cur iter: ", cur_iter
        for ind in range(len(y_list)):
            lstm_net.x_list_add(input_val_arr[ind])
            #print "y_pred[%d] : %f" % (ind, lstm_net.lstm_node_list[ind].state.h[0])

        loss = lstm_net.y_list_is(y_list, ToyLossLayer)
        #print "loss: ", loss
        losss.append(loss)
        lstm_param.apply_diff(lr=0.1)
        lstm_net.x_list_clear()
    print 'finish'
    print losss
    for ind in range(len(y_list)):
        lstm_net.x_list_add(input_val_arr[ind])
        print "y_pred[%d] : %f\t\t%f" % (ind, lstm_net.lstm_node_list[ind].state.h[0],y_train[ind])

    loss = lstm_net.y_list_is(y_list, ToyLossLayer)
    losss.append(loss)
    lstm_param.apply_diff(lr=0.1)
    lstm_net.x_list_clear()

In [302]:
# 直接用碱基序列
def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count 
    mem_cell_ct = 1000
    x_dim = 13
    concat_len = x_dim + mem_cell_ct
    lstm_param = LstmParam(mem_cell_ct, x_dim) 
    lstm_net = LstmNetwork(lstm_param)
    y_list =  y_train[:10]
    input_val_arr = [np.random.random(x_dim) for _ in y_list]
    input_val_arr =[ temp  for temp in X_train[:10,:]]
    print np.shape(input_val_arr)
    print input_val_arr[:3]
    
    for cur_iter in range(100):
        if cur_iter%5== 0:
            print "cur iter: ", cur_iter
        for ind in range(len(y_list)):
            lstm_net.x_list_add(input_val_arr[ind])
            if (ind%5==0)&(cur_iter%5==0):
               print "y_pred[%d] : %f\t%f\n" % (ind, lstm_net.lstm_node_list[ind].state.h[0],y_train[ind])

        loss = lstm_net.y_list_is(y_list, ToyLossLayer)
        if cur_iter%5==0:
            print "loss: ", loss
        lstm_param.apply_diff(lr=0.1)
        lstm_net.x_list_clear()
        
        


    print "cur iter: ", '1001'
    for ind in range(len(y_list)):
        lstm_net.x_list_add(input_val_arr[ind])
        print "y_pred[%d] : %f\t%f\n" % (ind, lstm_net.lstm_node_list[ind].state.h[0],y_train[ind])

    loss = lstm_net.y_list_is(y_list, ToyLossLayer)
    print "loss: ", loss
    lstm_param.apply_diff(lr=0.1)
    lstm_net.x_list_clear()

In [303]:
example_0()

(10, 13)
[array([ 1.10000002,  1.10000002,  1.10000002,  3.5       ,  1.10000002,
        5.5999999 ,  7.9000001 ,  5.5999999 ,  1.10000002,  5.5999999 ,
        7.9000001 ,  3.5       ,  7.9000001 ], dtype=float32), array([ 3.5       ,  3.5       ,  5.5999999 ,  5.5999999 ,  1.10000002,
        5.5999999 ,  1.10000002,  5.5999999 ,  7.9000001 ,  1.10000002,
        1.10000002,  5.5999999 ,  7.9000001 ], dtype=float32), array([ 1.10000002,  1.10000002,  1.10000002,  5.5999999 ,  3.5       ,
        7.9000001 ,  1.10000002,  5.5999999 ,  7.9000001 ,  3.5       ,
        7.9000001 ,  7.9000001 ,  7.9000001 ], dtype=float32)]
cur iter:  0
y_pred[0] : 0.479539	0.319000

y_pred[5] : 0.120963	0.000000

loss:  7.68951441284
cur iter:  5
y_pred[0] : -0.000000	0.319000

y_pred[5] : -0.000000	0.000000

loss:  0.779439977122
cur iter:  10
y_pred[0] : -0.000000	0.319000

y_pred[5] : -0.000000	0.000000

loss:  0.779439977122
cur iter:  15
y_pred[0] : -0.000000	0.319000

y_pred[5] : -0.000000	0.0000

In [ ]:
arra

In [286]:
def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count 
    mem_cell_ct = 1000
    x_dim = 50
    concat_len = x_dim + mem_cell_ct
    lstm_param = LstmParam(mem_cell_ct, x_dim) 
    lstm_net = LstmNetwork(lstm_param)
    y_list = [-0.5,0.2,0.1, -0.5]
    input_val_arr = [ np.floor(np.random.random(x_dim)) for _ in y_list]

    for cur_iter in range(500):
        print "cur iter: ", cur_iter
        for ind in range(len(y_list)):
            lstm_net.x_list_add(input_val_arr[ind])
            print "y_pred[%d] : %f" % (ind, lstm_net.lstm_node_list[ind].state.h[0])

        loss = lstm_net.y_list_is(y_list, ToyLossLayer)
        print "loss: ", loss
        lstm_param.apply_diff(lr=0.1)
        lstm_net.x_list_clear()